Build AI agents with tool calling, persistent memory, and MCP server integration — all defined declaratively as tables with computed columns.

This notebook walks through the full lifecycle described in the [Agents & MCP](https://docs.pixeltable.com/use-cases/agents-mcp) use case:

| Phase | What you'll build |
|-------|-------------------|
| **1. Tools** | UDF tools, query tools, tool registration |
| **2. Workflow** | Agent table, tool selection, execution, context assembly, final response |
| **3. MCP** | Connect to MCP servers, use external tools |
| **4. Memory** | Chat history, memory bank, semantic recall |
| **5. Deploy** | Flask/FastAPI integration with chat history |

**Key idea:** Instead of imperative control flow, define your agent as a table. Each row is a user query; computed columns define the reasoning chain. Pixeltable handles orchestration, caching, and persistence.

**Requirements:** `OPENAI_API_KEY` environment variable.

## Setup

In [1]:
%pip install -qU pixeltable openai sentence-transformers mcp

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
vllm 0.14.1 requires torch==2.9.1; platform_system == "Darwin" or platform_machine == "ppc64le" or platform_machine == "aarch64", but you have torch 2.10.0 which is incompatible.
fiftyone 1.13.0 requires sse-starlette<1,>=0.10.3, but you have sse-starlette 3.2.0 which is incompatible.

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import getpass
import os

if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key: ')

In [3]:
import datetime

import pixeltable as pxt
from pixeltable.functions import openai
from pixeltable.functions.huggingface import sentence_transformer

pxt.drop_dir('agents', force=True)
pxt.create_dir('agents')

/Users/pierre/pixeltable/pixeltable/env.py:494: UserWarning: Progress reporting is disabled because ipywidgets is not installed. To fix this, run: `pip install ipywidgets`
  warnings.warn(


Connected to Pixeltable database at: postgresql+psycopg://postgres:@/pixeltable?host=/Users/pjlb/.pixeltable/pgdata


W0219 13:20:28.727000 87265 site-packages/torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
/Users/pierre/pixeltable/pixeltable/catalog/table_version.py:670: PixeltableWarning: The computed column 'document_text' in table 'collection' is no longer valid.
The UDF 'functions.extract_document_text' cannot be located, because
the symbol 'functions.extract_document_text' is no longer a UDF. (Was the `@pxt.udf` decorator removed?)
You can continue to query existing data from this column, but evaluating it on new data will raise an error.
  col.init_value_expr(tvp)
/Users/pierre/pixeltable/pixeltable/catalog/table_version.py:670: PixeltableWarning: The computed column 'tool_output' in table 'tools' is no longer valid.
The UDF 'functions.get_latest_news' cannot be located, because
the symbol 'functions.get_latest_news' is no longer a UDF. (Was the `@pxt.udf` decorator removed?)
You can continue to query existing data fro

Created directory 'agents'.


### Build a Knowledge Base (for Agentic RAG)

Before building the agent, we need a knowledge base it can search. This mirrors the document pipeline from the [Backend for AI Apps](https://docs.pixeltable.com/use-cases/ai-applications) use case.

In [4]:
from pixeltable.functions.document import document_splitter

docs = pxt.create_table('agents.docs', {'document': pxt.Document, 'title': pxt.String})

base_url = 'https://raw.githubusercontent.com/pixeltable/pixeltable/main/docs/resources/rag-demo'
docs.insert([
    {'document': f'{base_url}/Argus-Market-Digest-June-2024.pdf', 'title': 'Argus Market Digest'},
    {'document': f'{base_url}/Company-Research-Alphabet.pdf', 'title': 'Alphabet Research'},
    {'document': f'{base_url}/Zacks-Nvidia-Report.pdf', 'title': 'Nvidia Report'},
])

chunks = pxt.create_view(
    'agents.chunks', docs,
    iterator=document_splitter(
        docs.document, separators='sentence,token_limit',
        limit=128, overlap=0, metadata='title,heading',
    ),
)

chunks.add_embedding_index(
    'text',
    string_embed=sentence_transformer.using(model_id='all-MiniLM-L6-v2'),
)

print(f'Knowledge base ready: {docs.count()} docs, {chunks.count()} chunks')

Created table 'docs'.
Inserted 3 rows with 0 errors in 0.01 s (265.85 rows/s)
Knowledge base ready: 3 docs, 527 chunks


---

## Phase 1 — Define Tools

Agents need tools. In Pixeltable, any `@pxt.udf` (external API calls, computations) or `@pxt.query` (database searches) can be registered as a tool the LLM can call.

### Tool UDFs: External API Calls

Wrap any Python code as a `@pxt.udf` tool. The LLM sees the function name, docstring, and parameter types.

In [5]:
@pxt.udf
def stock_lookup(ticker: str) -> str:
    """Look up the current stock price, daily change, and key metrics for a given ticker symbol."""
    data = {
        'GOOGL': 'Alphabet (GOOGL): $178.02 (+1.2%), P/E: 27.3, Market Cap: $2.2T',
        'NVDA': 'NVIDIA (NVDA): $135.58 (+3.1%), P/E: 65.2, Market Cap: $3.3T',
        'AAPL': 'Apple (AAPL): $213.25 (-0.4%), P/E: 33.1, Market Cap: $3.3T',
        'MSFT': 'Microsoft (MSFT): $448.30 (+0.8%), P/E: 37.5, Market Cap: $3.3T',
        'AMZN': 'Amazon (AMZN): $185.30 (+1.5%), P/E: 62.8, Market Cap: $1.9T',
    }
    return data.get(ticker.upper(), f'No data available for {ticker}')


@pxt.udf
def market_summary() -> str:
    """Get today's market summary including major index performance."""
    return (
        'Market Summary (June 21, 2024): '
        'DJIA 39,134.76 (+0.77%), S&P 500 5,482.87 (+0.25%), '
        'Nasdaq 17,721.59 (-0.18%). '
        'Tech rotation continued as Nvidia pulled back from highs. '
        'Broad-market breadth improved with financials and healthcare leading.'
    )

### Query Tools: Agentic RAG

`@pxt.query` functions turn semantic search into callable tools. The agent can decide *when* to search, *what* to search for, and use the results in its reasoning.

In [6]:
@pxt.query
def search_documents(query_text: str):
    """Search financial research documents by semantic similarity. Use this to find analysis, forecasts, and detailed company information."""
    sim = chunks.text.similarity(query_text)
    return (
        chunks
        .order_by(sim, asc=False)
        .select(text=chunks.text, source=chunks.title, score=sim)
        .limit(5)
    )

/var/folders/s4/0zdx499s6sv3_0jll6ccdbh00000gn/T/ipykernel_87265/1116462431.py:4: DeprecationWarning: Use of similarity() without specifying an explicit modality is deprecated -- since version 0.5.7. Please use one of the following instead:
  .similarity(string=...)
  .similarity(image=...)
  .similarity(audio=...)
  .similarity(video=...)
  sim = chunks.text.similarity(query_text)


In [7]:
# Verify the underlying search works
sim = chunks.text.similarity('Nvidia earnings and revenue')
chunks.order_by(sim, asc=False).select(chunks.text, source=chunks.title, sim=sim).limit(3).collect()

Error: () for an absolute path is invalid

### Register Tools

`pxt.tools()` bundles UDFs and queries into a single registry that gets passed to the LLM.

In [ ]:
tools = pxt.tools(
    stock_lookup,
    market_summary,
    search_documents,
)

print('Registered tools:', [t.name for t in tools])

---

## Phase 2 — Agent Workflow

The agent is a table. Each row is a user prompt. Computed columns define a reasoning chain:

```
prompt → [LLM selects tool] → [execute tool] → [LLM generates answer]
```

### Create Agent Table

In [ ]:
agent = pxt.create_table('agents.workflow', {
    'prompt': pxt.String,
    'system_prompt': pxt.String,
})

SYSTEM_PROMPT = (
    'You are a financial research assistant. You have access to tools for '
    'looking up stock prices, getting market summaries, and searching '
    'through financial research documents. Always use the most appropriate '
    'tool before answering. Use search_documents for analysis and forecasts, '
    'stock_lookup for current prices, and market_summary for broad market data.'
)

### Step 1: Tool Selection

The first LLM call sees the available tools and decides which one to call, with what arguments.

In [ ]:
agent.add_computed_column(
    tool_response=openai.chat_completions(
        messages=[
            {'role': 'system', 'content': agent.system_prompt},
            {'role': 'user', 'content': agent.prompt},
        ],
        model='gpt-4o-mini',
        tools=tools,
        tool_choice=tools.choice(required=True),
    )
)

### Step 2: Tool Execution

`invoke_tools()` automatically executes whichever tool the LLM selected and returns the results.

In [ ]:
agent.add_computed_column(
    tool_output=openai.invoke_tools(tools, agent.tool_response)
)

### Step 3: Final Response

A second LLM call takes the tool output and generates a polished, grounded answer.

In [ ]:
@pxt.udf
def assemble_context(prompt: str, tool_output: dict, system_prompt: str) -> list[dict]:
    """Combine the user prompt, tool output, and system prompt into final LLM messages."""
    tool_results = []
    for name, result in tool_output.items():
        if result is not None:
            tool_results.append(f'[{name}]: {result}')
    tool_str = '\n'.join(tool_results)

    return [
        {
            'role': 'system',
            'content': f'{system_prompt}\n\nYou have already called tools and received results. Use them to answer concisely.',
        },
        {
            'role': 'user',
            'content': f'Tool results:\n{tool_str}\n\nOriginal question: {prompt}',
        },
    ]

agent.add_computed_column(
    final_messages=assemble_context(agent.prompt, agent.tool_output, agent.system_prompt)
)

agent.add_computed_column(
    answer=openai.chat_completions(
        messages=agent.final_messages,
        model='gpt-4o-mini',
    ).choices[0].message.content
)

### Run the Agent

In [ ]:
agent.insert([
    {'prompt': 'What are analysts saying about Nvidia?', 'system_prompt': SYSTEM_PROMPT},
    {'prompt': 'Get me the stock price for GOOGL', 'system_prompt': SYSTEM_PROMPT},
    {'prompt': 'How did the market do today?', 'system_prompt': SYSTEM_PROMPT},
])

agent.select(agent.prompt, agent.answer).collect()

### Inspect Tool Usage

See exactly which tool the LLM called and what it returned.

In [ ]:
agent.select(agent.prompt, agent.tool_output).collect()

---

## Phase 3 — MCP Integration

The [Model Context Protocol (MCP)](https://modelcontextprotocol.io/) is an open standard for connecting LLMs to external tools. Pixeltable can load tools from any MCP server and use them alongside local UDFs.

### Connect to an MCP Server

`pxt.mcp_udfs()` connects to any MCP-compliant server and returns the tools as callable UDFs.

In [ ]:
# Connect to the Pixeltable documentation MCP server
mcp_tools = pxt.mcp_udfs('https://docs.pixeltable.com/mcp')

for tool in mcp_tools:
    print(f'  {tool.name}: {tool.comment()[:80]}...')

In [ ]:
# Bundle MCP tools for LLM use, alongside local tools
mcp_toolset = pxt.tools(*mcp_tools)

# Create a table with MCP tool-calling pipeline
mcp_agent = pxt.create_table('agents.mcp_agent', {'query': pxt.String})

mcp_agent.add_computed_column(
    response=openai.chat_completions(
        messages=[{'role': 'user', 'content': mcp_agent.query}],
        model='gpt-4o-mini',
        tools=mcp_toolset,
    )
)

mcp_agent.add_computed_column(
    tool_results=openai.invoke_tools(mcp_toolset, mcp_agent.response)
)

In [ ]:
mcp_agent.insert([
    {'query': 'How do I create an embedding index in Pixeltable?'},
    {'query': 'What LLM providers does Pixeltable support?'},
])

mcp_agent.select(mcp_agent.query, mcp_agent.tool_results).collect()

### Combine Local + MCP Tools

Mix tools from any source — local UDFs, query functions, and MCP servers — in a single agent.

In [ ]:
combined_tools = pxt.tools(
    stock_lookup,
    search_documents,
    *mcp_tools,
)

print('Combined tools:', [t.name for t in combined_tools])

### Build Your Own MCP Server

Expose Pixeltable tables as MCP tools for Claude Desktop, Cursor, or any MCP client.

```python
# Save as mcp_server.py and run: python mcp_server.py
from mcp.server.fastmcp import FastMCP
import pixeltable as pxt

mcp = FastMCP('FinancialResearch', stateless_http=True)

@mcp.tool()
def search_financial_docs(query: str) -> str:
    """Search financial research documents."""
    chunks = pxt.get_table('agents.chunks')
    sim = chunks.text.similarity(query)
    results = chunks.order_by(sim, asc=False).limit(5).collect()
    return '\n---\n'.join(r['text'] for r in results)

@mcp.tool()
def get_stock_price(ticker: str) -> str:
    """Get a stock price."""
    # Your implementation here
    pass

if __name__ == '__main__':
    mcp.run(transport='streamable-http')
```

---

## Phase 4 — Memory

Agents need memory. Pixeltable tables with embedding indexes provide both recency-based and semantic recall — persisted across sessions.

### Chat History

Store every conversation turn with timestamps and user IDs. An embedding index enables semantic search over all past conversations.

In [ ]:
chat_history = pxt.create_table('agents.chat_history', {
    'role': pxt.String,
    'content': pxt.String,
    'timestamp': pxt.Timestamp,
    'user_id': pxt.String,
})

chat_history.add_embedding_index(
    'content',
    string_embed=sentence_transformer.using(model_id='all-MiniLM-L6-v2'),
)

In [ ]:
now = datetime.datetime.now

chat_history.insert([
    {'role': 'user', 'content': 'What is the Nvidia outlook?', 'timestamp': now(), 'user_id': 'alice'},
    {'role': 'assistant', 'content': 'Analysts are bullish on NVDA due to strong AI chip demand. Revenue grew 262% YoY.', 'timestamp': now(), 'user_id': 'alice'},
    {'role': 'user', 'content': 'How about Alphabet cloud revenue?', 'timestamp': now(), 'user_id': 'alice'},
    {'role': 'assistant', 'content': 'Google Cloud grew 28% YoY to $9.57B in Q1 2024, driven by AI workloads.', 'timestamp': now(), 'user_id': 'alice'},
    {'role': 'user', 'content': 'Compare AAPL and MSFT earnings.', 'timestamp': now(), 'user_id': 'bob'},
    {'role': 'assistant', 'content': 'Apple reported $90.8B revenue (+5% YoY). Microsoft reported $61.9B (+17% YoY), led by Azure growth.', 'timestamp': now(), 'user_id': 'bob'},
])

### Recent History Query

Retrieve the most recent turns for a given user — standard recency-based context.

In [ ]:
@pxt.query
def get_recent_history(user_id: str, limit: int = 4):
    """Get the most recent chat messages for a user."""
    return (
        chat_history
        .where(chat_history.user_id == user_id)
        .order_by(chat_history.timestamp, asc=False)
        .select(role=chat_history.role, content=chat_history.content)
        .limit(limit)
    )

# Test recent history inline
(
    chat_history
    .where(chat_history.user_id == 'alice')
    .order_by(chat_history.timestamp, asc=False)
    .select(chat_history.role, chat_history.content)
    .limit(4)
    .collect()
)

### Semantic Recall

Search past conversations by meaning — find relevant context even from old conversations.

In [ ]:
@pxt.query
def recall_memory(query_text: str, user_id: str, limit: int = 3):
    """Search past conversations by semantic similarity for a specific user."""
    sim = chat_history.content.similarity(query_text)
    return (
        chat_history
        .where(chat_history.user_id == user_id)
        .order_by(sim, asc=False)
        .select(role=chat_history.role, content=chat_history.content, score=sim)
        .limit(limit)
    )

# Alice asks about cloud — semantic search finds the Alphabet conversation
sim = chat_history.content.similarity('cloud computing growth')
(
    chat_history
    .where(chat_history.user_id == 'alice')
    .order_by(sim, asc=False)
    .select(chat_history.role, chat_history.content, score=sim)
    .limit(3)
    .collect()
)

### Memory Bank

Beyond chat history, store explicit facts, code snippets, or user preferences that the agent should remember.

In [ ]:
memory_bank = pxt.create_table('agents.memory_bank', {
    'content': pxt.String,
    'memory_type': pxt.String,
    'context': pxt.String,
    'timestamp': pxt.Timestamp,
    'user_id': pxt.String,
})

memory_bank.add_embedding_index(
    'content',
    string_embed=sentence_transformer.using(model_id='all-MiniLM-L6-v2'),
)

memory_bank.insert([
    {
        'content': 'User prefers concise answers with specific numbers.',
        'memory_type': 'preference',
        'context': 'Inferred from interaction style',
        'timestamp': now(),
        'user_id': 'alice',
    },
    {
        'content': 'User is tracking NVDA, GOOGL, and MSFT for a portfolio review.',
        'memory_type': 'fact',
        'context': 'User mentioned portfolio in earlier conversation',
        'timestamp': now(),
        'user_id': 'alice',
    },
    {
        'content': 'User wants weekly market digests emailed every Friday.',
        'memory_type': 'preference',
        'context': 'User configuration request',
        'timestamp': now(),
        'user_id': 'bob',
    },
])

In [ ]:
@pxt.query
def search_memory(query_text: str, user_id: str, limit: int = 3):
    """Search the memory bank for saved facts, preferences, and context about a user."""
    sim = memory_bank.content.similarity(query_text)
    return (
        memory_bank
        .where(memory_bank.user_id == user_id)
        .order_by(sim, asc=False)
        .select(
            content=memory_bank.content,
            memory_type=memory_bank.memory_type,
            score=sim,
        )
        .limit(limit)
    )

# Search Alice's memory bank for portfolio-related facts
sim = memory_bank.content.similarity('portfolio stocks')
(
    memory_bank
    .where(memory_bank.user_id == 'alice')
    .order_by(sim, asc=False)
    .select(memory_bank.content, memory_bank.memory_type, score=sim)
    .limit(3)
    .collect()
)

---

## Phase 5 — Deploy

Wire the agent, memory, and tools together in an API endpoint.

### Flask Endpoint with Memory

```python
from flask import Flask, request, jsonify
from datetime import datetime
import pixeltable as pxt

app = Flask(__name__)
agent = pxt.get_table('agents.workflow')
chat_history = pxt.get_table('agents.chat_history')

SYSTEM_PROMPT = 'You are a financial research assistant with access to tools.'

@app.route('/chat', methods=['POST'])
def chat():
    data = request.json
    user_id = data['user_id']
    prompt = data['message']

    # 1. Store user message
    chat_history.insert([{
        'role': 'user',
        'content': prompt,
        'timestamp': datetime.now(),
        'user_id': user_id,
    }])

    # 2. Trigger agent workflow (computed columns run automatically)
    agent.insert([{
        'prompt': prompt,
        'system_prompt': SYSTEM_PROMPT,
    }])

    # 3. Get the answer
    result = agent.order_by(agent._rowid, asc=False).select(agent.answer).limit(1).collect()
    answer = result[0]['answer']

    # 4. Store assistant response
    chat_history.insert([{
        'role': 'assistant',
        'content': answer,
        'timestamp': datetime.now(),
        'user_id': user_id,
    }])

    return jsonify({'response': answer})
```

---

## Full Schema Overview

In [ ]:
print('=== Knowledge Base ===')
print(f'docs: {docs.count()} rows')
print(f'chunks: {chunks.count()} rows')

print('\n=== Agent Workflow ===')
print(f'agent: {agent.count()} rows')
agent.describe()

print('\n=== Memory ===')
print(f'chat_history: {chat_history.count()} rows')
print(f'memory_bank: {memory_bank.count()} rows')

---

## Summary

| Phase | What Pixeltable handled |
|-------|------------------------|
| **Tools** | `@pxt.udf` for API calls, `@pxt.query` for agentic RAG, `pxt.tools()` for registration |
| **Workflow** | Agent as a table — tool selection, execution, and synthesis as computed columns |
| **MCP** | `pxt.mcp_udfs()` to load external tools, mix with local tools |
| **Memory** | Chat history + memory bank, both with embedding indexes for semantic recall |
| **Deploy** | Flask endpoint — `insert()` triggers the full agent pipeline |

### What you didn't need

- An agent framework (LangChain, CrewAI, AutoGen) — computed columns are the orchestration
- A separate memory store (Redis, Zep) — Pixeltable tables with embedding indexes
- A vector database for RAG — built-in embedding indexes
- Imperative control flow — the agent is declarative, each row triggers the full chain
- Manual tool dispatch — `invoke_tools()` handles execution automatically

### Next steps

| Topic | Link |
|-------|------|
| Tool calling deep dive | [Tool Calling Cookbook](https://docs.pixeltable.com/howto/cookbooks/agents/llm-tool-calling) |
| Agent memory patterns | [Agent Memory](https://docs.pixeltable.com/howto/cookbooks/agents/pattern-agent-memory) |
| RAG pipeline patterns | [RAG Pipeline](https://docs.pixeltable.com/howto/cookbooks/agents/pattern-rag-pipeline) |
| Pixelbot (full agent app) | [GitHub](https://github.com/pixeltable/pixelbot) |
| Pixelagent framework | [GitHub](https://github.com/pixeltable/pixelagent) |
| MCP server for AI IDEs | [GitHub](https://github.com/pixeltable/mcp-server-pixeltable-developer) |

In [ ]:
pxt.drop_dir('agents', force=True)